#### Імпорт бібліотек

In [17]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats
from sklearn.metrics import r2_score

#### Дані за варіантом

In [2]:
data = {
    'Y': [14.85, 11.94, 8.03, 7.11, 9.50, 9.40, 11.60, 8.14, 15.62, 11.12, 7.34, 10.58, 7.37, 10.63, 10.63],
    'X1': [60, 48, 39, 28, 45, 37, 58, 27, 58, 47, 38, 44, 23, 57, 38],
    'X2': [30, 19, 8, 18, 9, 23, 15, 17, 28, 16, 7, 15, 25, 8, 24],
    'X3': [0.15, 0.02, 0.14, 0.11, 0.12, 0.10, 0.13, 0.09, 0.07, 0.12, 0.08, 0.11, 0.15, 0.13, 0.07],
    'X4': [5.0, 3.1, 4.7, 2.5, 4.9, 2.6, 4.6, 3.4, 4.8, 4.9, 3.2, 4.7, 2.7, 5.0, 2.9]
}
df = pd.DataFrame(data)

#### 1. Визначення статистичних оцінок параметрів лінійної множинної регресії

In [5]:
Y = df['Y']
X = df[['X1', 'X2', 'X3', 'X4']]
X = sm.add_constant(X) # Додаємо константу для B0
X_array = X.values # Перетворюємо DataFrame X на NumPy array
feature_names = ['X1', 'X2', 'X3', 'X4']

model = sm.OLS(Y, X).fit()
beta_hat = model.params

print(f"B0* = {beta_hat['const']:.4f}")
print(f"B1* = {beta_hat['X1']:.4f}")
print(f"B2* = {beta_hat['X2']:.4f}")
print(f"B3* = {beta_hat['X3']:.4f}")
print(f"B4* = {beta_hat['X4']:.4f}\n")

print("Отримане рівняння регресії:")
print(f"Y = {beta_hat['const']:.4f} + {beta_hat['X1']:.4f}*X1 + {beta_hat['X2']:.4f}*X2 + {beta_hat['X3']:.4f}*X3 + {beta_hat['X4']:.4f}*X4\n")

B0* = -0.0002
B1* = 0.1113
B2* = 0.1983
B3* = -19.7984
B4* = 1.0403

Отримане рівняння регресії:
Y = -0.0002 + 0.1113*X1 + 0.1983*X2 + -19.7984*X3 + 1.0403*X4



#### 2. Побудова довірчого інтервалу для функції регресії з надійністю γ = 0,99

In [16]:
alpha = 1 - 0.99
n = len(Y)
p = X_array.shape[1] # Кількість параметрів (включаючи константу)
degrees_of_freedom = n - p
t_critical = stats.t.ppf(1 - alpha / 2, degrees_of_freedom)

# Прогнозування значень Y
y_predicted = model.predict(X)

# Розрахунок середньоквадратичної помилки (MSE)
mse = model.mse_resid

# Розрахунок стандартних помилок прогнозованих значень
pred_var = mse * np.diag(X_array @ np.linalg.inv(X_array.T @ X_array) @X_array.T)
pred_se = np.sqrt(pred_var)

# Розрахунок меж довірчого інтервалу
confidence_interval_lower = y_predicted - t_critical * pred_se
confidence_interval_upper = y_predicted + t_critical * pred_se

print("Довірчий інтервал для функції регресії (γ = 0.99):\n")
for i in range(n):
    features = [f"{feature_names[j]}={df.iloc[i][feature_names[j]]:.2f}" for j in range(len(feature_names))]
    features_str = ", ".join(features)
    print(f"Для спостереження {i+1} (вектор ознак: [{features_str}]): [{confidence_interval_lower[i]:.4f}, "
          f"{confidence_interval_upper[i]:.4f}]")
print("\n")

Довірчий інтервал для функції регресії (γ = 0.99):

Для спостереження 1 (вектор ознак: [X1=60.00, X2=30.00, X3=0.15, X4=5.00]): [14.7914, 14.9259]
Для спостереження 2 (вектор ознак: [X1=48.00, X2=19.00, X3=0.02, X4=3.10]): [11.8729, 12.0045]
Для спостереження 3 (вектор ознак: [X1=39.00, X2=8.00, X3=0.14, X4=4.70]): [7.9956, 8.0928]
Для спостереження 4 (вектор ознак: [X1=28.00, X2=18.00, X3=0.11, X4=2.50]): [7.0627, 7.1548]
Для спостереження 5 (вектор ознак: [X1=45.00, X2=9.00, X3=0.12, X4=4.90]): [9.4712, 9.5574]
Для спостереження 6 (вектор ознак: [X1=37.00, X2=23.00, X3=0.10, X4=2.60]): [9.3562, 9.4517]
Для спостереження 7 (вектор ознак: [X1=58.00, X2=15.00, X3=0.13, X4=4.60]): [11.5877, 11.6940]
Для спостереження 8 (вектор ознак: [X1=27.00, X2=17.00, X3=0.09, X4=3.40]): [8.0775, 8.1853]
Для спостереження 9 (вектор ознак: [X1=58.00, X2=28.00, X3=0.07, X4=4.80]): [15.5516, 15.6788]
Для спостереження 10 (вектор ознак: [X1=47.00, X2=16.00, X3=0.12, X4=4.90]): [11.0875, 11.1629]
Для спост

#### 3. Обчислення коефіцієнта множинної кореляції

In [20]:
r = np.sqrt(r2_score(Y, y_predicted))
print(f"Коефіцієнт множинної кореляції R = {r:.4f}\n")
print(f"Коефіцієнт детермінації R2 = {r2_score(Y, y_predicted)}")

Коефіцієнт множинної кореляції R = 1.0000

Коефіцієнт детермінації R2 = 0.9999103761748839


In [29]:
df['Y_pred'] = beta_hat['const'] + beta_hat['X1'] * df['X1'] + beta_hat['X2'] * df['X2'] + beta_hat['X3'] * df['X3'] + beta_hat['X4'] * df['X4']

In [33]:
df['Error (Y - Y_pred)'] = (np.abs(df['Y'] - df['Y_pred']))

In [34]:
df

,Y,X1,X2,X3,X4,Y_pred,Error (Y - Y_pred)
0,14.85,60,30,0.15,5.0,14.858658,0.008658
1,11.94,48,19,0.02,3.1,11.938721,0.001279
2,8.03,39,8,0.14,4.7,8.044236,0.014236
3,7.11,28,18,0.11,2.5,7.108718,0.001282
4,9.50,45,9,0.12,4.9,9.514291,0.014291
5,9.40,37,23,0.10,2.6,9.403944,0.003944
6,11.60,58,15,0.13,4.6,11.640870,0.040870
7,8.14,27,17,0.09,3.4,8.131378,0.008622
8,15.62,58,28,0.07,4.8,15.615225,0.004775
9,11.12,47,16,0.12,4.9,11.125212,0.005212
